# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [52]:
import pandas as pd
import numpy as np
from scipy import stats
import math
from statsmodels.stats.proportion import proportions_ztest as pz

In [53]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [54]:
# number of callbacks for black-sounding names
call_b = sum(data[data.race=='b'].call)
call_w = sum(data[data.race=='w'].call)
print('callbacks for the black group: ', call_b)
print('callbacks for the white group: ', call_w)

callbacks for the black group:  157.0
callbacks for the white group:  235.0


In [55]:
# Taking a look at the number of each group
num_b = len(data[data.race=='b'])
num_w = len(data[data.race=='w'])
print('Number of where race is b :', num_b)
print('Number of where race is w :', num_w)

Number of where race is b : 2435
Number of where race is w : 2435


In [56]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

### 1) What test is appropriate for this problem? Does CLT apply?

For this we can compare the call back rates between the two sub groups (b and w). As we have a large enough sample (> 4K) the CTL will apply for the difference of two proportions. Thoguh we are dealing with a binary (call / no-call) parameter CTL will still apply.

Further, as we will be looking at a difference between 'expected' and 'observed' frequencies a chi-sqaure ($X^2$) test will be appropriate.

### 2) What are the null and alternate hypotheses?

h0: P of the two groups is equal

h1: P of the balck subgroup is < that of the white subgroup

In [58]:
prob_b = sum(data[data.race=='b'].call) / len(data[data.race=='b'])
prob_w = sum(data[data.race=='w'].call) / len(data[data.race=='w'])

print('The callback rate for the black group was ', round(prob_b, 4))
print('The callback rate for the white group was ', round(prob_w, 4))
print('The difference was: ', (prob_w - prob_b))

The callback rate for the black group was  0.0645
The callback rate for the white group was  0.0965
The difference was:  0.0320328542094


### 3) Compute margin of error, confidence interval, and p-value.

Here I will be useing the Z statistic and a 95% condidence interval.

In [59]:
# Calculating the margin of error
z = 1.96
error = z * math.sqrt( ( prob_w*(1-prob_w) / num_b) + (prob_b*(1-prob_b)/num_w) )

print('Error = ', round(error, 5))

Error =  0.01526


In [60]:
# Calculating the 95% confidence interval
low, high = [round((prob_w-prob_b-z*error),4), round((prob_w-prob_b+z*error), 4)]
print('The confidence interval is given by :', low, 'and', high)

The confidence interval is given by : 0.0021 and 0.0619


In [61]:
#calculating the p-value
pz(np.array([call_b, call_w]), np.array([num_b, num_w]), value=0)

(-4.1084121524343464, 3.9838868375850767e-05)

As the *p* is extremply small we can reject the null and sat that there is a statistically significcant difference in the call back rates between the two groups. 

### 4) Write a story describing the statistical significance in the context or the original problem.

Given the investigations above we are 95% confident that those with white sounding names receive between **0.0021** and **0.0619** more callback than those with black sounding names. The practical meaning of this is that though we have a statistical difference in the proportion of call backs the magnitude of the difference is minimal. This leads me to believe that there is more to the overall difference in call back rates than race. 

### 5) Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis shows that racial intonation of names has a statistically significant effect on callbacks rates. However there are many other factors that would need consideration such as education, experience, and technological skills. A more indepth analysis would be needed in order to determine the significance of these other factors and a comparison with race/name. If we were able to control for these factors and still found a difference in the call back rate then there would be a much stonger link.